## Generate expected results for back tests

This notebook generates results for backtests. These include

* Backtests for run of simulation with default parameters (`test_backtest.py`).
* Backtests for runs of simulation where you are determining an appropriate number of replications (`test_backtest_replications.py`).

They save the relevant results dataframe as `.csv`, and use these to verify that the model produces consistent results.

The run time is provided at the end of this notebook.

## Set-up

Load required packages.

In [1]:
# pylint: disable=missing-module-docstring
# To ensure any updates to `simulation/` are fetched without needing to restart
# the notebook environment, reload `simulation/` before execution of each cell
%load_ext autoreload
%autoreload 1
%aimport simulation

In [2]:
# pylint: disable=wrong-import-position
# Import required packages
import os
import time
from IPython.display import display

from simulation.model import Param, Runner, run_scenarios
from simulation.replications import confidence_interval_method

Start timer.

In [3]:
# Start timer
start_time = time.time()

Define path to expected results.

In [4]:
# Define path to folder for expected results for tests
TESTS = '../tests/exp_results/'

## Run of simulation with default parameters

In [5]:
# Define model parameters
param = Param(
    patient_inter=4,
    mean_n_consult_time = 10,
    number_of_nurses = 4,
    warm_up_period = 500,
    data_collection_period = 1500,
    number_of_runs=5,
    audit_interval = 50,
    scenario_name = 0,
    cores = 1
)

# Run the replications
experiment = Runner(param)
experiment.run_reps()

In [6]:
# Patient-level results
display(experiment.patient_results_df)
experiment.patient_results_df.to_csv(
    os.path.join(TESTS, 'patient.csv'), index=False)

,patient_id,arrival_time,q_time_nurse,time_with_nurse,run,q_time_unseen_nurse
0,1,504.513444,0.0,1.664896,0,NaN
1,2,507.230909,0.0,29.462508,0,NaN
2,3,508.394374,0.0,7.345594,0,NaN
3,4,512.975372,0.0,7.986073,0,NaN
4,5,521.933116,0.0,7.127312,0,NaN
...,...,...,...,...,...,...
1885,365,1978.104484,0.0,1.712059,4,NaN
1886,366,1980.171724,0.0,11.023619,4,NaN
1887,367,1986.506388,0.0,8.212316,4,NaN
1888,368,1988.648731,0.0,24.898160,4,NaN


In [7]:
# Run results
display(experiment.run_results_df)
experiment.run_results_df.to_csv(
    os.path.join(TESTS, 'run.csv'), index=False)

,run_number,scenario,arrivals,mean_q_time_nurse,mean_time_with_nurse,mean_nurse_utilisation,mean_nurse_utilisation_tw,mean_nurse_q_length,count_nurse_unseen,mean_q_time_nurse_unseen
0,0,0,403,2.043788,9.577436,0.642332,0.642332,0.549098,0,NaN
1,1,0,382,1.255117,10.208371,0.646797,0.646797,0.319636,0,NaN
2,2,0,369,3.392586,10.731510,0.668010,0.668010,0.835793,0,NaN
3,3,0,367,2.227175,10.523889,0.639573,0.639573,0.544915,0,NaN
4,4,0,369,1.653496,9.809751,0.601042,0.601042,0.406760,0,NaN


In [8]:
# Interval audit results
display(experiment.interval_audit_df)
experiment.interval_audit_df.to_csv(
    os.path.join(TESTS, 'interval.csv'), index=False)

,resource_name,simulation_time,utilisation,queue_length,running_mean_wait_time,run
0,nurse,500,0.50,0,0.258217,0
1,nurse,550,0.00,0,0.237723,0
2,nurse,600,0.00,0,0.220244,0
3,nurse,650,1.00,0,0.215357,0
4,nurse,700,1.00,1,1.677251,0
...,...,...,...,...,...,...
145,nurse,1750,1.00,0,2.228919,4
146,nurse,1800,0.50,0,2.174401,4
147,nurse,1850,1.00,1,2.116596,4
148,nurse,1900,0.75,0,2.121569,4


In [9]:
# Overall results
display(experiment.overall_results_df)
experiment.overall_results_df.to_csv(
    os.path.join(TESTS, 'overall.csv'), index=True)

,arrivals,mean_q_time_nurse,mean_time_with_nurse,mean_nurse_utilisation,mean_nurse_utilisation_tw,mean_nurse_q_length,count_nurse_unseen,mean_q_time_nurse_unseen
mean,378.000000,2.114432,10.170191,0.639551,0.639551,0.531240,0.0,NaN
std_dev,15.198684,0.806577,0.480342,0.024255,0.024255,0.195914,0.0,NaN
lower_95_ci,359.128341,1.112935,9.573768,0.609435,0.609435,0.287981,0.0,NaN
upper_95_ci,396.871659,3.115930,10.766615,0.669667,0.669667,0.774500,0.0,NaN


## Scenario analysis

In [10]:
# Run scenarios
param = Param(
    patient_inter=4,
    mean_n_consult_time=10,
    number_of_nurses=5,
    warm_up_period=2000,
    data_collection_period=6000,
    number_of_runs=3,
    audit_interval=120,
    cores=1
)
scenario_results = run_scenarios(
    scenarios={'patient_inter': [3, 4],
               'number_of_nurses': [6, 7]},
    param=param
)

# Preview
display(scenario_results.head())

# Save to csv
scenario_results.to_csv(os.path.join(TESTS, 'scenario.csv'), index=True)

There are 4 scenarios. Running:
{'patient_inter': 3, 'number_of_nurses': 6}
{'patient_inter': 3, 'number_of_nurses': 7}
{'patient_inter': 4, 'number_of_nurses': 6}
{'patient_inter': 4, 'number_of_nurses': 7}


,run_number,scenario,arrivals,mean_q_time_nurse,mean_time_with_nurse,mean_nurse_utilisation,mean_nurse_utilisation_tw,mean_nurse_q_length,count_nurse_unseen,mean_q_time_nurse_unseen,patient_inter,number_of_nurses
0,0,0,2004,0.486680,9.925934,0.553955,0.553955,0.162551,0,NaN,3,6
1,1,0,1993,0.746175,10.377197,0.573874,0.573874,0.247854,0,NaN,3,6
2,2,0,2017,0.386324,9.856724,0.553419,0.553419,0.129869,0,NaN,3,6
0,0,1,2004,0.121476,9.925934,0.474819,0.474819,0.040573,0,NaN,3,7
1,1,1,1993,0.226682,10.377197,0.491892,0.491892,0.075296,0,NaN,3,7


## Running the simulation when attempting to determine an appropriate number of parameters

Each of these should return the same output dataframe:

```
_, man_df_simple = confidence_interval_method_simple(
    replications=20, metric='mean_time_with_nurse')
```

```
_, man_df = confidence_interval_method(
    replications=20, metric='mean_time_with_nurse')
```

```
observer = ReplicationTabulizer()
analyser = ReplicationsAlgorithm(
    verbose=False,
    observer=observer,
    initial_replications=20,
    replication_budget=20)
_ = analyser.select(runner=Runner(Param()), metric='mean_time_with_nurse')
auto_df = observer.summary_table().head(20)
```

Hence, we will just run one, but will compare all against it in backtests.

In [11]:
# Specify the parameters for this back test (so remains consistent even if
# defaults used are changed)
param = Param(
    patient_inter=4,
    mean_n_consult_time=10,
    number_of_nurses=5,
    warm_up_period=1440*13,
    data_collection_period=1440*30,
    number_of_runs=31,
    audit_interval=120,
    scenario_name=0,
    cores=1
)

_, man_df = confidence_interval_method(
    replications=40,
    metrics=['mean_time_with_nurse',
             'mean_q_time_nurse',
             'mean_nurse_utilisation'],
    param=param)

display(man_df)

man_df.to_csv(
    os.path.join(TESTS, 'replications.csv'), index=False)

,replications,data,cumulative_mean,stdev,lower_ci,upper_ci,deviation,metric
0,1,9.842380,9.842380,NaN,NaN,NaN,NaN,mean_time_with_nurse
1,2,10.060424,9.951402,NaN,NaN,NaN,NaN,mean_time_with_nurse
2,3,9.925090,9.942631,0.110075,9.669189,10.216073,0.027502,mean_time_with_nurse
3,4,9.938504,9.941599,0.089900,9.798549,10.084650,0.014389,mean_time_with_nurse
4,5,10.016611,9.956602,0.084775,9.851339,10.061864,0.010572,mean_time_with_nurse
...,...,...,...,...,...,...,...,...
35,36,0.493083,0.498001,0.007179,0.495572,0.500430,0.004877,mean_nurse_utilisation
36,37,0.508147,0.498275,0.007272,0.495851,0.500700,0.004866,mean_nurse_utilisation
37,38,0.502386,0.498383,0.007204,0.496015,0.500751,0.004751,mean_nurse_utilisation
38,39,0.499974,0.498424,0.007113,0.496118,0.500730,0.004626,mean_nurse_utilisation


## Run time

In [12]:
# Get run time in seconds
end_time = time.time()
runtime = round(end_time - start_time)

# Display converted to minutes and seconds
print(f'Notebook run time: {runtime // 60}m {runtime % 60}s')

Notebook run time: 0m 7s
